<a href="https://colab.research.google.com/github/matthewpecsok/data_engineering/blob/main/tutorials/de_tutorial_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Extraction from a OLTP Source System

In this tutorial we'll begin exploring the concept of querying a OLTP database, not for analytics, but rather to start extracting data for downstream data engineering applications. 

**Data Engineering**

**Matthew Pecsok 2/10/2023**




# 1.&nbsp;A quick overview of lists and tuples

in python a list is object that can store multiple items, is changeable, and allows duplicate values. 

A list can begin empty and items can be added to it. 

In [130]:
my_list = [] #create an empty list
my_list

[]

In [131]:
my_list.append('movie_1') # append a single item to the list
my_list

['movie_1']

In [132]:
my_list.extend(['movie_2','movie_3']) # concatenate two lists together creating a single list as a result
my_list

['movie_1', 'movie_2', 'movie_3']

In [133]:
len(my_list) # how long is out new list?

3

What if we want to store more complex information? We can use a **tuple**. A tuple is similar to a list except it is ordered unchangeable, and allows duplicate values. Notice the parenthesis instead of brackets.

In [134]:
movie_1 = ('Toy Story',1995)
movie_1

('Toy Story', 1995)

In [135]:
movie_2 = ('Monsters Inc.',2001)
movie_2

('Monsters Inc.', 2001)

In [136]:
my_complex_list = []
my_complex_list

[]

In [137]:
my_complex_list.extend([movie_1,movie_2])

In [138]:
my_complex_list

[('Toy Story', 1995), ('Monsters Inc.', 2001)]

In [139]:
my_complex_list[0] # get the zeroth (or first depending on how you count) element in the list

('Toy Story', 1995)

In [140]:
my_complex_list[0][1]

1995

## For loops on lists

It's quite easy to loop through a list and execute some code on that list. Here's a few examples to get your comfortable. 

Please note, for loops are not the most efficient way to accomplish tasks like this, but they are simple to understand. 

In [194]:
i = 0 

for fruit in ['apples','oranges','bananas']:
  print(f'the current fruit is {fruit}')
  i += 1

print(f'we looped {i} times')

the current fruit is apples
the current fruit is oranges
the current fruit is bananas
we looped 3 times


In [196]:
for the_number in [1,2,3,4,5,6,42]:
  print(f'{the_number} is even = {(the_number%2==0)}')

1 is even = False
2 is even = True
3 is even = False
4 is even = True
5 is even = False
6 is even = True
42 is even = True


The takeaway here is that a for loop allows us to iterate over a list and execute code for each element in the list.

# 2.&nbsp;Package import and get database

In [141]:
import pandas as pd
import sqlite3
import tqdm

In [142]:
!wget -qO movies.db https://github.com/matthewpecsok/data_engineering/blob/main/data/movies.sqlite?raw=true

con = sqlite3.connect('movies.db')

%load_ext sql
%sql sqlite:///movies.db

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [143]:
con # con is our connection to the database

In [144]:
cur = con.cursor()
cur # cursor 

In [145]:
pd.read_sql_query('select * from movies limit 2',con)

,id,original_title,budget,popularity,release_date,revenue,title,vote_average,vote_count,overview,tagline,uid,director_id
0,43597,Avatar,237000000,150,2009-12-10,2787965087,Avatar,7.2,11800,"In the 22nd century, a paraplegic Marine is di...",Enter the World of Pandora.,19995,4762
1,43598,Pirates of the Caribbean: At World's End,300000000,139,2007-05-19,961000000,Pirates of the Caribbean: At World's End,6.9,4500,"Captain Barbossa, long believed to be dead, ha...","At the end of the world, the adventure begins.",285,4763


In [146]:
res = cur.execute('select * from movies').fetchall()
len(res)

4773

In [147]:
release_date_range = cur.execute("select min(release_date),max(release_date),JULIANDAY(max(release_date))-JULIANDAY(min(release_date)) as days from movies").fetchall()
release_date_range

[('1916-09-04', '2017-02-03', 36677.0)]

If we create an extraction for each date (as we might in a daily batch system) then we will have 36677 batches. 

to visualize this process let's print out just the first 4 batches

In [148]:
for release_date in range(0,4):
  result = cur.execute(f"select * from movies where release_date = date('1916-09-04','+{release_date} days')").fetchall()
  display(result)

[(48189,
  'Intolerance',
  385907,
  3,
  '1916-09-04',
  8394751,
  'Intolerance',
  7.4,
  60,
  'The story of a poor young woman, separated by prejudice from her husband and baby, is interwoven with tales of intolerance from throughout history.',
  'The Cruel Hand of Intolerance',
  3059,
  6968)]

[]

[]

[]

In [149]:
movie_list = [(48189,
  'Intolerance',
  385907,
  3,
  '1916-09-04',
  8394751,
  'Intolerance',
  7.4,
  60,
  'The story of a poor young woman, separated by prejudice from her husband and baby, is interwoven with tales of intolerance from throughout history.',
  'The Cruel Hand of Intolerance',
  3059,
  6968)]

movie_list[0][1] # the first movie in the list and second column

'Intolerance'

as you can see the first batch has data, but the next 3 do not. 

Let's see how long it takes to run all 36,677 batch queries.

tqdm gives us a running counter for our for loop

In [150]:
for release_date_int in tqdm.tqdm(range(0,36678)):
  batch = cur.execute(f"select * from movies where release_date = date('1916-09-04','+{release_date_int} days')").fetchall()

100%|██████████| 36678/36678 [00:53<00:00, 689.54it/s]


We will simulate a daily pull by writing a simple for loop and processing all of the movies for each date. In a real system you can imagine this process kicks off each day using the previous day's date to extract all of the movies entered each day.

The extraction appears to take about a minute or so. 

**Of course, we haven't moved the data anywhere yet so that will take additional time.**

Let's create a new database with a NEW table that just has the batch number and the movie title as we migrate the data from the existing to this new table. 

This NEW database is EMPTY (for now)

In [151]:
con_migrate = sqlite3.connect('movies_migrated.db')
cur_migrate = con_migrate.cursor()

In [152]:
cur_migrate.execute("drop table if exists movie_migration").fetchall()
cur_migrate.execute("create table movie_migration(batch_id INT,movie_title TEXT)").fetchall()

[]

whenever you are about to run something 36678 times, start with just a few rows to make sure it works as expected.

In [153]:
for release_date_int in tqdm.tqdm(range(0,3)): 
  batch = cur.execute(f"select title from movies where release_date = date('1916-09-04','+{release_date_int} days')").fetchall()
  for i in range(len(batch)): # there may be multiple movies for a given release date, loop for each
    display(batch[i])
    cur_migrate.execute(f"insert into movie_migration (batch_id,movie_title) values ({release_date_int},'{batch[i][0]}')")

  0%|          | 0/3 [00:00<?, ?it/s]

('Intolerance',)

100%|██████████| 3/3 [00:00<00:00, 191.36it/s]


In [154]:
pd.read_sql_query('select * from movie_migration',con_migrate)

,batch_id,movie_title
0,0,Intolerance


Great! It worked and inserted the one movie we expected and didn't seem to create any other rows we wouldn't have wanted.

Next let's clean the table up and get rid of the rows before re-running.

In [155]:
cur_migrate.execute(f"delete from movie_migration")
cur_migrate.execute(f"select count(1) from movie_migration").fetchall()

[(0,)]

Now run it for ALL records.

This process creates an insert statement for EACH DAY. Each batch is then extremely small (often zero records). 

# 3.&nbsp;Small batch insert

In [156]:
for release_date_int in tqdm.tqdm(range(0,36678)): 
  batch = cur.execute(f"select title from movies where release_date = date('1916-09-04','+{release_date_int} days')").fetchall()
  for i in range(len(batch)):
    # display(batch[i]) uncomment this to see the tuple of movies that are being inserted
    cur_migrate.execute(f"""insert into movie_migration (batch_id,movie_title) values ({release_date_int},"{batch[i][0]}")""")

100%|██████████| 36678/36678 [00:49<00:00, 741.02it/s]


That took a little over a minute to run. Let's check out our new database and table

In [174]:
pd.read_sql_query('select * from movie_migration limit 8',con_migrate)

,batch_id,movie_title
0,1916,Intolerance
1,1925,The Big Parade
2,1927,Metropolis
3,1929,Pandora's Box
4,1929,The Broadway Melody
5,1930,Hell's Angels
6,1932,A Farewell to Arms
7,1933,42nd Street


In [158]:
pd.read_sql_query('select count(1) from movie_migration',con_migrate)

,count(1)
0,4773


In [159]:
pd.read_sql_query('select count(1) from movies',con)

,count(1)
0,4773


# 4.&nbsp;Large batch insert

Create batches by YEAR instead of by day. This greatly increases the batch size. When we processed by day we had over 36,000 batches, many with 0 records. With year we should have only 100 or so batches with many more records in each batch.






*   By Year



In [160]:
%%sql select 

round(avg(count),1) as avg_yearly_release_count,
avg(count) as avg_release_count

from (
  select count(1) as count from movies group by strftime('%Y',release_date)
  ) as count

 * sqlite:///movies.db


,avg_yearly_release_count,avg_release_count
0,53.0,53.033333




*   By Date (day)



In [161]:
%%sql select 
round(avg(count),1) as avg_daily_release_count,
avg(count) as avg_release_count

from (
  select count(1) as count from movies group by release_date
  ) as count

 * sqlite:///movies.db


,avg_daily_release_count,avg_release_count
0,1.5,1.458741


the average number of movies released each year is 53. 

The table below attempts to show the tradeoffs between larger and smaller batches. 

Batch Type | Batch Count | Avg Movies per Batch | Pro | Con
--|--|--- | ---| ---
by Day | 36,000  | 1.5 | low memory batches | many batches 
by Year | 100  | 53 | higher memory batches | fewer batches


In [168]:
cur_migrate.execute(f"delete from movie_migration")
cur_migrate.execute(f"select count(1) from movie_migration").fetchall()

[(0,)]

In [179]:
%%sql select 
min(strftime('%Y',release_date)) as first_year,
max(strftime('%Y',release_date)) as last_year,

cast(max(strftime('%Y',release_date)) as int) - cast(min(strftime('%Y',release_date)) as int) as total_years
 from movies

 * sqlite:///movies.db


,first_year,last_year,total_years
0,1916,2017,101


In [172]:
for release_year_int in tqdm.tqdm(range(1916,2018)): 
  batch = cur.execute(f"select {release_year_int},title from movies where strftime('%Y',release_date) = cast({release_year_int} as str)").fetchall()
  cur_migrate.executemany(f"insert into movie_migration (batch_id,movie_title) values (?,?)",batch)

con_migrate.commit()

100%|██████████| 102/102 [00:00<00:00, 343.22it/s]


this new batch mode went from 45 seconds to less than a second to insert all the records. 

In [173]:
pd.read_sql_query("select count(1) from movie_migration",con_migrate)

,count(1)
0,4773


In [206]:
pd.read_sql_query("select * from movie_migration order by batch_id desc limit 10",con_migrate)

,batch_id,movie_title
0,2017,Growing Up Smith
1,2016,Batman v Superman: Dawn of Justice
2,2016,Captain America: Civil War
3,2016,Star Trek Beyond
4,2016,The Legend of Tarzan
5,2016,X-Men: Apocalypse
6,2016,Suicide Squad
7,2016,The Jungle Book
8,2016,Independence Day: Resurgence
9,2016,Shin Godzilla


# 5.&nbsp;Fixed size batch insert

Create batches by a specific number of rows. We then have a specific batch size each time (except for the last batch). In this way we create a specific payload size for each batch rather than a variable size as shown in the previous examples. 


In [235]:
cur_migrate.execute(f"delete from movie_migration")
cur_migrate.execute(f"select count(1) from movie_migration").fetchall()

[(0,)]

In [225]:
cur.execute(f"select count(1) from movies").fetchall()

[(4773,)]

4773 movies. in batches of 1000 that should be 5 batches. 

In [226]:
cur.execute(f"select cast(cast(count(1) as float)/1000 as float) from movies").fetchall()

[(4.773,)]

In [231]:
fixed_batch = [0, 1000, 2000, 3000, 4000]
fixed_batch

[0, 1000, 2000, 3000, 4000]

In this case we take advantage of the limit and offset parameters in sql to create batches of fixed size (1000 rows) in each batch. 

Fixed batch sizes allows for predictable payload and memory requirements. 

**LIMIT** forces a specific number of rows to be returned.

**OFFSET** sets the number of rows to skip. 

By keeping limit fixed and increasing OFFSET we can pull

row range| batch | OFFSET | LIMIT
--|--| -- | --
1-1000 | (batch 1) | 0 | 1000
1001-2000 | (batch 2) | 1000 | 1000
2001-3000 | (batch 3) | 2000 | 1000
3001-4000 | (batch 4) | 3000 | 1000
4001-5000  | (batch 5) | 4000  | 1000


In [236]:
for batch in tqdm.tqdm(fixed_batch): 
  batch = cur.execute(f"select {batch},title from movies LIMIT 1000 OFFSET {batch}").fetchall()
  cur_migrate.executemany(f"insert into movie_migration (batch_id,movie_title) values (?,?)",batch)

con_migrate.commit()

100%|██████████| 5/5 [00:00<00:00, 204.30it/s]


In [237]:
pd.read_sql_query("select count(1),batch_id from movie_migration group by batch_id",con_migrate)

,count(1),batch_id
0,1000,0
1,1000,1000
2,1000,2000
3,1000,3000
4,773,4000


In [221]:
%%sql select 
count(1),batch_id from movie_migration group by batch_id

 * sqlite:///movies.db
(sqlite3.OperationalError) no such table: movie_migration
[SQL: select 
count(1),batch_id from movie_migration group by batch_id]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


# 6.&nbsp;Conclusion

Moving data between two databases means that we need to break our data up into groups of rows aka batches. Trying to do one enormous batch would generally overwhelm our system resources. 

* **Small Batches**: By choosing many small batches we reduce memory requirements for each batch, but we increase the number of batches which if latency is a concern may be a problem.

*  **Large Batches**: By choosing large batches we increase memory requirements for each batch, but we decrease the total number of trips to the database which can reduce the impact of latency. 

* **Fixed Batches**: can force a specific number of rows into the payload. This can help create a standardized data pull for each batch rather than a variable size as the previous two example showed. 